In [1]:
# Import relevant modules

%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import copy

import pandas as pd
import numpy as np
import healpy as hp
from scipy.interpolate import interp1d
from scipy.integrate import quad
from scipy.optimize import minimize
from astropy.io import fits
from tqdm import *
import matplotlib.pyplot as plt

# NPTFit modules
from NPTFit import nptfit # module for performing scan
from NPTFit import create_mask as cm # module for creating the mask
from NPTFit import psf_correction as pc # module for determining the PSF correction

In [2]:
FermiData = np.load('fermi_data/fermidata_counts.npy').astype(np.int32)
fermi_exposure = np.load('fermi_data/fermidata_exposure.npy')
dif = np.load('fermi_data/template_dif.npy')
iso = np.load('fermi_data/template_iso.npy')
psc = np.load('fermi_data/template_psc.npy')
subhalos = np.load('EinastoTemplate.npy')
subhalos = subhalos/np.sum(subhalos)

In [3]:
n = nptfit.NPTF(tag='norm')
n.load_data(FermiData, fermi_exposure)

pscmask=np.array(np.load('fermi_data/fermidata_pscmask.npy'), dtype=bool)
mask = cm.make_mask_total(band_mask = True, band_mask_range = 5, mask_ring = True, inner = 20, outer = 180, custom_mask = pscmask)
n.load_mask(mask)

n.add_template(dif, 'dif')
n.add_template(iso, 'iso')
n.add_template(psc, 'psc')

n.add_poiss_model('dif', '$A_\mathrm{dif}$', [0,10], False)
n.add_poiss_model('iso', '$A_\mathrm{iso}$', [0,20], False)
n.add_poiss_model('psc', '$A_\mathrm{psc}$', [0,10], False)

xsec0 = 1e-24
A0 = 10**(-13.2551)
n10 = -0.38119
n20 = 10
Fb0 = 10**(-9.42857)

xsec_arr = np.logspace(-60, -22, 5)

LL_xsec_ary = np.zeros(len(xsec_arr))

for ix, xsec in enumerate(xsec_arr):
    A = A0 * xsec/xsec0
    Fb = Fb0 * xsec0/xsec

    new_n = copy.copy(n)
    new_n.add_template(subhalos, 'subhalos')
    #new_n.add_poiss_model('subhalos', '$A_\mathrm{sub}$', fixed=True)
    new_n.add_non_poiss_model('subhalos', 
                           ['$A^\mathrm{ps}_\mathrm{iso}$','$n_1$','$n_2$','$F_b$'],
                           fixed_params = [[0, A], [1, n10], [2, n20], [3,Fb]])
    new_n.configure_for_scan()
    scpy_min = minimize(lambda x: -new_n.ll(x) ,x0=[1.,1.,1.], bounds=[[0,30.], [0,30.], [0,30.]], options={'disp': False, 'ftol':1e-12}, method='SLSQP')
    max_LL = -scpy_min['fun']
    best_fit_params = scpy_min['x']
    print("Template best-fit params are", best_fit_params)
    print("Max L-L is", max_LL)
    LL_xsec_ary[ix] = max_LL

The number of parameters to be fit is 3


/home/jean/miniconda3/envs/py36/lib/python3.6/site-packages/NPTFit-0.2-py3.6-linux-x86_64.egg/NPTFit/nptf_scan.py:445: RuntimeWarning: divide by zero encountered in longdouble_scalars
  self.masked_compressed_data_expreg[i])


Template best-fit params are [1. 1. 1.]
Max L-L is -1763540269434905.8
The number of parameters to be fit is 3
Template best-fit params are [1. 1. 1.]
Max L-L is -1763540269434905.8
The number of parameters to be fit is 3
Template best-fit params are [1. 1. 1.]
Max L-L is -1763540269434905.8
The number of parameters to be fit is 3
Template best-fit params are [1. 1. 1.]
Max L-L is -1763540269434905.8
The number of parameters to be fit is 3
Template best-fit params are [1. 1. 1.]
Max L-L is -1763540269434905.8
